In [15]:
meetings = Schedule.objects.distinct().values_list('meeting', flat=True).order_by('meeting')
meetings, meetings[0]

(<QuerySet [1, 3, 4, 5, 6, 7, 9, 11, 12, 14, 15, 16, 18, 19, 20, 22, 23, 24, 25, 26, '...(remaining elements truncated)...']>,
 1)

In [23]:
Schedule.objects.update(pair_index=-1)

548

In [24]:
from collections import defaultdict
pairs_by_id = {}
pair_index = 0
for sch in Schedule.objects.filter(meeting=meetings[0]).order_by('player__id'):
    player_id, partner_id, pidx = sch.player.id, sch.partner.id, sch.pair_index
    if pidx > -1:
        pairs_by_id[player_id] = pidx
        pairs_by_id[partner_id] = pidx
    else:
        if player_id in pairs_by_id:
            sch.pair_index = pairs_by_id[player_id]
            sch.save()
            pairs_by_id[partner_id] = pairs_by_id[player_id]
            print(f"Updated {player_id} to {pair_index}")
        elif partner_id in pairs_by_id:
            sch.pair_index = pairs_by_id[partner_id]
            sch.save()
            pairs_by_id[player_id] = pairs_by_id[partner_id]
        else:
            # Add the value to both
            pairs_by_id[player_id] = pair_index
            pairs_by_id[partner_id] = pair_index
            print(f"set {player_id} and {partner_id} to index {pair_index}")
            sch.pair_index = pair_index
            sch.save()
            pair_index += 1
        
        
    
    
    
    

set 1 and 2 to index 0
Updated 2 to 1
set 4 and 54 to index 1
set 9 and 93 to index 2
set 19 and 105 to index 3
Updated 54 to 4
set 74 and 114 to index 4
set 80 and 94 to index 5
Updated 93 to 6
Updated 94 to 6
Updated 105 to 6
set 108 and 109 to index 6
Updated 109 to 7
set 112 and 113 to index 7
Updated 113 to 8
Updated 114 to 8


In [38]:
m = 41
couples = Schedule.objects.filter(meeting=m).distinct('pair_index').order_by('pair_index')
couples[0].player.id, couples[0].partner.id

(2, 1)

In [40]:
[(c.player.id, c.partner.id, c.pair_index) for c in couples]

[(2, 1, 0),
 (17, 18, 1),
 (111, 19, 2),
 (40, 87, 3),
 (99, 54, 4),
 (61, 66, 5),
 (113, 112, 6),
 (135, 134, 7)]

In [42]:
from django.db.models import Max
Schedule.objects.filter(meeting=m).aggregate(Max('pair_index'))

{'pair_index__max': 7}